
# Simple dose computation

In this example we create a generic CT and use the MCsquare dose calculator
to compute the dose image and plot dose-volume histograms (DVH).


Imports



In [ ]:
import os
import math
import numpy as np
from matplotlib import pyplot as plt

# Try to install opentps if running in Colab or IPython
try:
    get_ipython
    # Only run if package not already installed
    try:
        import opentps
    except ImportError:
        get_ipython().run_line_magic("pip", "install opentps")
except NameError:
    # Not running in IPython (e.g., Sphinx Gallery) – assume opentps is preinstalled
    pass

Import OpenTPS modules



In [ ]:
from opentps.core.data.images import CTImage, ROIMask
from opentps.core.data.plan import ProtonPlanDesign
from opentps.core.data import DVH, Patient
from opentps.core.io import mcsquareIO
from opentps.core.io.scannerReader import readScanner
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.protons.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.processing.imageProcessing.resampler3D import resampleImage3DOnImage3D, resampleImage3D

## Create generic CT



In [ ]:
ctCalibration = readScanner(DoseCalculationConfig().scannerFolder)
bdl = mcsquareIO.readBDL(DoseCalculationConfig().bdlFile)

patient = Patient()
patient.name = 'Patient'

ctSize = 150
huAir = -1024.
huWater = ctCalibration.convertRSP2HU(1.)

ct = CTImage()
ct.name = 'CT'
ct.patient = patient

data = huAir * np.ones((ctSize, ctSize, ctSize))
data[:, 50:, :] = huWater
ct.imageArray = data

## Create ROI



In [ ]:
roi = ROIMask()
roi.patient = patient
roi.name = 'TV'
roi.color = (255, 0, 0)  # red
data = np.zeros((ctSize, ctSize, ctSize), dtype=bool)
data[65:85, 65:85, 65:85] = True
roi.imageArray = data

## Configure MCsquare



In [ ]:
mc2 = MCsquareDoseCalculator()
mc2.beamModel = bdl
mc2.ctCalibration = ctCalibration
mc2.nbPrimaries = 1e7

## Build treatment plan



In [ ]:
beamNames = ["Beam1", "Beam2", "Beam3"]
gantryAngles = [0., 90., 270.]
couchAngles = [0., 0., 0.]

planDesign = ProtonPlanDesign()
planDesign.ct = ct
planDesign.targetMask = roi
planDesign.gantryAngles = gantryAngles
planDesign.beamNames = beamNames
planDesign.couchAngles = couchAngles
planDesign.calibration = ctCalibration
planDesign.spotSpacing = 5.0
planDesign.layerSpacing = 5.0
planDesign.targetMargin = 5.0

plan = planDesign.buildPlan()
plan.PlanName = "NewPlan"

## Plot CT with ROI



In [ ]:
roi = resampleImage3DOnImage3D(roi, ct)
COM_coord = roi.centerOfMass
COM_index = roi.getVoxelIndexFromPosition(COM_coord)
Z_coord = COM_index[2]

img_ct = ct.imageArray[:, :, Z_coord].transpose(1, 0)
contourTargetMask = roi.getBinaryContourMask()
img_mask = contourTargetMask.imageArray[:, :, Z_coord].transpose(1, 0)

output_path = 'Output'
if not os.path.exists(output_path):
    os.makedirs(output_path)

plt.imshow(img_ct, cmap='Blues')
plt.colorbar()
plt.contour(img_mask, colors="red")
plt.title("Created CT with ROI")
plt.text(5, 40, "Air", color='black')
plt.text(5, 100, "Water", color='white')
plt.text(71, 77, "TV", color='red')
plt.savefig(os.path.join(output_path, 'SimpleCT.png'), format='png')
plt.show()

## Compute dose



In [ ]:
doseImage = mc2.computeDose(ct, plan)

## Plot dose and DVH



In [ ]:
img_dose = resampleImage3DOnImage3D(doseImage, ct)
img_dose = img_dose.imageArray[:, :, Z_coord].transpose(1, 0)

scoringSpacing = [2, 2, 2]
scoringGridSize = [int(math.floor(i / j * k)) for i, j, k in zip([150, 150, 150], scoringSpacing, [1, 1, 1])]
roiResampled = resampleImage3D(roi, origin=ct.origin, gridSize=scoringGridSize, spacing=scoringSpacing)
target_DVH = DVH(roiResampled, doseImage)

fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(img_ct, cmap='gray')
ax[0].imshow(img_mask, alpha=.2, cmap='binary')
dose = ax[0].imshow(img_dose, cmap='jet', alpha=.2)
cbar = plt.colorbar(dose, ax=ax[0])
cbar.set_label('Dose (Gy)')
ax[1].plot(target_DVH.histogram[0], target_DVH.histogram[1], label=target_DVH.name)
ax[1].set_xlabel("Dose (Gy)")
ax[1].set_ylabel("Volume (%)")
ax[1].grid(True)
ax[1].legend()
plt.savefig(os.path.join(output_path, 'SimpleDose.png'), format='png')
plt.show()

Print DVH summary



In [ ]:
print(f"D95 = {target_DVH.D95} Gy")
print(f"D5 = {target_DVH.D5} Gy")
print(f"D5 - D95 = {target_DVH.D5 - target_DVH.D95} Gy")